In [4]:
import requests

res = requests.post('https://api-seller.ozon.ru/v2/category/tree', headers={
    'Client-Id': '589954', 'Api-Key': '2e0076d1-1ecd-4105-b1df-d87e1a244648'
})

arr = res.json()['result']
categories = []


def parse(arr):
    for obj in arr:
        tmp = {}
        tmp['category_id'] = obj['category_id']
        tmp['title'] = obj['title']
        if obj['children'] and len(obj['children']) != 0:
            parse(obj['children'])
        else:
            categories.append(tmp)


parse(arr)

print(categories[:10])


[{'category_id': 99584951, 'title': 'Цифровой контент - Пополнение баланса социальной сети'}, {'category_id': 17038150, 'title': 'ПО - Вспомогательная программа'}, {'category_id': 99583585, 'title': 'Цифровой контент - Отслеживание ребенка'}, {'category_id': 93227883, 'title': 'EP + Blu-ray'}, {'category_id': 17033823, 'title': 'CD + DVD'}, {'category_id': 71774050, 'title': 'Видеоигра'}, {'category_id': 17035755, 'title': 'Дистрибутив - Энциклопедия'}, {'category_id': 17033445, 'title': 'DVD с дополнениями'}, {'category_id': 1000003870, 'title': 'Транспортная карта Стрелка, Тройка'}, {'category_id': 17036871, 'title': '3D Blu-ray'}]


In [13]:
%pip install beautifulsoup4
%pip install selenium
%pip install pyppeteer

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 83.4/83.4 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 101.4/101.4 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python310\\Scripts\\pyppeteer-install.exe' -> 'c:\\Python310\\Scripts\\pyppeteer-install.exe.deleteme'


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import asyncio
from bs4 import BeautifulSoup
from pyppeteer import launch
import os
from collections import deque 

browser = await launch()
page = await browser.newPage()
await page.setUserAgent('Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0')

base = 'https://www.ozon.ru'
categories_url = 'https://www.ozon.ru/category/'
used_links = {categories_url}
found_links = set()


In [3]:

await page.goto(categories_url, {'waitUntil': 'networkidle0'})
page_content = await page.content()

def grab_categories(page_content):
    soup = BeautifulSoup(page_content)
    links = set()
    for a in soup.find_all('a', href=True):
        if re.fullmatch('\/category\/[^\/]+\\d+\/', a['href']) is not None:
            link = base + a['href']
            if link in found_links:
                continue

            print("Found the URL:", link)
            links.add(link)
            found_links.add(link)

    return links


links = grab_categories(page_content)


queue = deque(list(links))

while len(queue) != 0:
    url = queue.popleft()
    if url in used_links:
        continue
    used_links.add(url)
    try:
        await page.goto(url, {'waitUntil': 'networkidle0'})
        page_content = await page.content()
        links = grab_categories(page_content)
        queue.extend(list(links))
    except TimeoutError:
        print('timeout error!')
        queue.extend(url)



Found the URL: https://www.ozon.ru/category/supermarket-25000/
Found the URL: https://www.ozon.ru/category/elektronika-15500/
Found the URL: https://www.ozon.ru/category/zhenskaya-odezhda-7501/
Found the URL: https://www.ozon.ru/category/dom-i-sad-14500/
Found the URL: https://www.ozon.ru/category/detskie-tovary-7000/
Found the URL: https://www.ozon.ru/category/odezhda-obuv-i-aksessuary-7500/
Found the URL: https://www.ozon.ru/category/apteka-6000/
Found the URL: https://www.ozon.ru/category/bytovaya-tehnika-10500/
Found the URL: https://www.ozon.ru/category/knigi-16500/
Found the URL: https://www.ozon.ru/category/stroitelstvo-i-remont-9700/
Found the URL: https://www.ozon.ru/category/krasota-i-zdorove-6500/
Found the URL: https://www.ozon.ru/category/sport-i-otdyh-11000/
Found the URL: https://www.ozon.ru/category/tovary-dlya-zhivotnyh-12300/
Found the URL: https://www.ozon.ru/category/produkty-pitaniya-9200/
Found the URL: https://www.ozon.ru/category/avtotovary-8500/
Found the URL: 

In [ ]:
await browser.close()